# Home task: pandas 

## Question 1

- Load the energy data from the file [Energy Indicators.xls](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls).
It is a list of indicators of energy supply and renewable electricity production from the United Nations for the year 2013.


- It should be put into a DataFrame with the variable name of "energy"


- Make sure to exclude the footer and header information from the datafile.


- The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:<br>
`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']`


- Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule).


- For all countries which have missing data (e.g. data with `...`) make sure this is reflected as `np.NaN` values.


- Rename the following list of countries (for use in later questions):
    - `Republic of Korea`: `South Korea`,
    - `United States of America`: `United States`,
    - `United Kingdom of Great Britain and Northern Ireland`: `United Kingdom`,
    - `China, Hong Kong Special Administrative Region`: `Hong Kong`


- There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, e.g.:
    - `Bolivia (Plurinational State of)` should be `Bolivia`,
    - `Switzerland17` should be `Switzerland`.


- Next, load the GDP data from the file ["world_bank.csv"](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). 
It is a csv containing countries' GDP from 1960 to 2015 from World Bank. Call this DataFrame "GDP"


- Make sure to skip the header, and rename the following list of countries:
    - `Korea, Rep.`: `South Korea`,
    - `Iran, Islamic Rep.`: `Iran`,
    - `Hong Kong SAR, China`: `Hong Kong`


- Finally, load the "Sciamgo Journal and Country Rank data for [Energy Engineering and Power Technology"](http://www.scimagojr.com/countryrank.php?category=2102). It ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame "ScimEn"



- Join the three datasets: Energy, GDP, and ScimEn into a new dataset (using the intersection of country names). Use only the 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15).


- The index of this DataFrame should be the name of the country, and the columns should be<br>
`['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', 2012', '2013', '2014', '2015']`

Function "answer_one" should return the resulted DataFrame (20 columns and 15 entries)

## Answer the following questions in the context of only the top 15 countries by Scimagojr Rank (aka the DataFrame returned by `answer_one()`)

In [73]:
import pandas as pd
import requests
import numpy as np
#import re

#remove numbers and '(-//-)' from Country-column
def clean(value):
    value = str(value)
    value = value.replace(') ', ' (')
    value = value.split(' (')
    
    try:
        value = value[0] + value[1]
    except:
        value = value[0]
    value = value.replace('(', '')
    value = value.replace(')', '')
    value = value.replace('0', '')
    value = value.replace('1', '')
    value = value.replace('2', '')
    value = value.replace('3', '')
    value = value.replace('4', '')
    value = value.replace('5', '')
    value = value.replace('6', '')
    value = value.replace('7', '')
    value = value.replace('8', '')
    value = value.replace('9', '')
    #print(value)
    return value




def answer_one():

    #load energy data
    url = 'http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls'
    file = requests.get(url).content
    energy = pd.read_excel(file, skipfooter=38, skiprows=16)


    #rename columns & lil more
    energy = energy.drop(energy.columns[[0, 1]], axis=1)
    energy.rename(columns = {'Unnamed: 2':'Country',
                         'Energy Supply':'Energy_Supply',
                         'Energy Supply per capita':'Energy_Supply_per_Capita',
                         'Renewable Electricity Production':'%_Renewable'}, inplace = True)
    energy.iloc[0, 1] = 'Gigajouls'
    energy.loc[1:, 'Energy_Supply'] *= 10**6


    #quering for null-values
    for feature in energy.columns:
        energy[feature] = energy[feature].where(cond=(~energy[feature].isnull()), other=np.NaN)
    energy['Energy_Supply'].mask(energy['Energy_Supply'].str.contains('...') == True, np.NaN, inplace = True)
    energy['Energy_Supply_per_Capita'].mask(energy['Energy_Supply_per_Capita'].str.contains('...') == True, np.NaN, inplace = True)
    energy['%_Renewable'].mask(energy['%_Renewable'].str.contains('...') == True, np.NaN, inplace = True)


    #restore the first row
    energy.iloc[0, 0] = '-'
    energy.iloc[0, 1] = 'Gigajouls'
    energy.iloc[0, 2] = 'Gigajouls'
  

    #rename some countries
    energy.Country = energy.Country.where(cond=~(energy.Country=='Republic of Korea'), other='South Korea')
    energy.iloc[217, 0] = 'United States'
    energy.iloc[215, 0] = 'United Kingdom'
    energy.Country = energy.Country.where(cond=~(energy.Country=='China, Hong Kong Special Administrative Region'), other='Hong Kong')
    energy.iloc[99, 0] = 'Iran'


    #remove numbers and '(-//-)' from Country-column
    energy.Country = energy.Country.apply(clean)


    #load GDP data
    GDP = pd.read_csv('GDP.csv', sep=',', skiprows=4 )


    #rename some countries in GDP dataframe
    GDP['Country Name'] = GDP['Country Name'].where(cond=~(GDP['Country Name']=='Korea, Rep.'), other='South Korea')
    GDP['Country Name'] = GDP['Country Name'].where(cond=~(GDP['Country Name']=='Iran, Islamic Rep.'), other='Iran')
    GDP['Country Name'] = GDP['Country Name'].where(cond=~(GDP['Country Name']=='Hong Kong SAR, China'), other='Hong Kong')
    GDP = GDP.drop(columns=['Unnamed: 66']) #drop column


    #load Sciamgo Journal and Country Rank data
    ScimEn = pd.read_excel('scimagojr_rank.xlsx')
    ScimEn = ScimEn[ScimEn.Rank <=15]


    #drop columns that are not in 2006-2015 from GDP
    for feature in list(GDP.iloc[:,4:].columns):
        if feature != 'Country Name':
            if feature != 'Country Code':
                if feature != 'Indicator Name':
                    if feature != 'Indicator Code':
                        if not (int(feature)<2016 and int(feature)>2005):
                            GDP = GDP.drop([feature], axis=1)



    #MERGE
    dataframe = pd.merge(ScimEn, energy, left_on='Country', right_on='Country')
    dataframe.drop(columns=['Region'], inplace=True)
    dataframe = pd.merge(dataframe, GDP, left_on='Country', right_on='Country Name',  )
    dataframe.drop(columns=[ 'Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'], inplace=True)
    dataframe.rename(columns={'Energy_Supply':'Energy Supply',
                           'Energy_Supply_per_Capita':'Energy Supply per Capita',
                           '%_Rename':'% Rename'}, inplace=True)
    
    dataframe.set_index('Country', inplace=True)


    return dataframe
answer_one()

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,%_Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
China,1,303064,301778,3036531,2092737,10.02,273,127191000000,93,19.75491,2.752132e+12,3.550343e+12,4.594307e+12,5.101703e+12,6.087164e+12,7.551500e+12,8.532230e+12,9.570406e+12,1.047568e+13,1.106155e+13
United States,2,184851,181106,2623922,819242,14.19,389,90838000000,286,11.57098,1.381559e+13,1.447423e+13,1.476986e+13,1.447806e+13,1.504896e+13,1.559973e+13,1.625397e+13,1.684319e+13,1.755068e+13,1.820602e+13
India,3,60257,58589,590570,213760,9.80,198,33195000000,26,14.96908,9.402599e+11,1.216735e+12,1.198896e+12,1.341887e+12,1.675615e+12,1.823050e+12,1.827638e+12,1.856722e+12,2.039127e+12,2.103588e+12
Japan,4,52780,52281,557023,132113,10.55,203,18984000000,149,10.23282,4.601663e+12,4.579751e+12,5.106679e+12,5.289493e+12,5.759072e+12,6.233147e+12,6.272363e+12,5.212328e+12,4.896994e+12,4.444931e+12
United Kingdom,5,47141,45928,748994,132737,15.89,244,7920000000,124,10.60047,2.717060e+12,3.106182e+12,2.938882e+12,2.425798e+12,2.491110e+12,2.674891e+12,2.719158e+12,2.803291e+12,3.087166e+12,2.956574e+12
Germany,6,42343,41464,528645,115432,12.48,211,13261000000,165,17.90153,2.994704e+12,3.425578e+12,3.745264e+12,3.411261e+12,3.399668e+12,3.749315e+12,3.527143e+12,3.733805e+12,3.889093e+12,3.357586e+12
Russian Federation,7,39424,39189,142937,67935,3.63,96,30709000000,214,17.28868,9.899305e+11,1.299706e+12,1.660846e+12,1.222644e+12,1.524917e+12,2.045926e+12,2.208296e+12,2.292473e+12,2.059242e+12,1.363481e+12
Canada,8,35588,34940,665415,113363,18.70,246,10431000000,296,61.94543,1.319265e+12,1.468820e+12,1.552990e+12,1.374625e+12,1.617343e+12,1.793327e+12,1.828366e+12,1.846597e+12,1.805750e+12,1.556509e+12
Italy,9,31260,29959,433388,105641,13.86,177,6530000000,109,33.66723,1.949552e+12,2.213102e+12,2.408655e+12,2.199929e+12,2.136100e+12,2.294994e+12,2.086958e+12,2.141924e+12,2.162010e+12,1.836638e+12


### Question 2
What is the average GDP over the last 10 years for each country? (exclude missing values from this calculation.)

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [57]:
def answer_two():
    Top15 = answer_one()
    Series = Top15.iloc[0:, 10:].mean(axis=1).squeeze()
    
    return Series


In [74]:
answer_two()

Country
China                 6.927702e+12
United States         1.570403e+13
India                 1.602352e+12
Japan                 5.239642e+12
United Kingdom        2.792011e+12
Germany               3.523342e+12
Russian Federation    1.666746e+12
Canada                1.616359e+12
Italy                 2.142986e+12
South Korea           1.221372e+12
France                2.691337e+12
Iran                  4.563261e+11
Spain                 1.400727e+12
Brazil                1.988889e+12
Australia             1.207106e+12
dtype: float64

### Question 3
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [75]:
def answer_three():
    Top15 = answer_one()
    Top15['mean GDP'] = answer_two()
    Top15.sort_values(by=['mean GDP'], inplace=True)
    value = Top15['2015'].iloc[5]/Top15['2006'].iloc[5]
    return value

answer_three()

1.1798304668641362

### Question 4

Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [83]:
def answer_four():
    Top15 = answer_one()
    Top15['Ratio'] = Top15['Self-citations']/Top15['Citations']
    value = Top15['Ratio'].max()
    index = Top15[Top15['Ratio'] == value].index[0]
    return (index, value)
  

answer_four()

('China', 0.689186772669207)

### Question 5

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return a single string value.*

In [84]:
def answer_five():
    Top15 = answer_one()
    Top15['Population'] = Top15['Energy Supply']/Top15['Energy Supply per Capita']
    Top15.sort_values(by=['Population'], inplace=True)
    return Top15.index[2]

answer_five()

'Spain'

### Question 6
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*


In [121]:
def answer_six():
    Top15 = answer_one()
    Top15['Citable docs per capita'] = Top15['Citable documents']/(Top15['Energy Supply']/Top15['Energy Supply per Capita'])
    Top15['Citable docs per capita'] = pd.to_numeric(Top15['Citable docs per capita'])
    Top15['Energy Supply per Capita'] = pd.to_numeric(Top15['Energy Supply per Capita'])
    new = Top15.corr()
    value = new.loc['Citable docs per capita', 'Energy Supply per Capita']
    return value
answer_six()

0.7272331912434824

### Question 7
Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [176]:
def answer_seven():
    Top15 = answer_one()
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
    #Create Continent-column
    Top15['Country'] = Top15.index
    Top15['Continent'] = Top15.Country.apply(lambda x: ContinentDict.get(x))
    Top15.drop(columns=['Country'], inplace=True)

    #Create new dataframe
    Top15['Population'] = Top15['Energy Supply']/ Top15['Energy Supply per Capita']
    dataframe = pd.DataFrame(columns = ['size', 'sum', 'mean', 'std'])
    for continent, miniframe in Top15.groupby(['Continent']):
        dataframe.loc[continent] = [len(miniframe), miniframe['Population'].sum(),
                                    miniframe['Population'].mean(),
                                    miniframe['Population'].std()]



    return dataframe

answer_seven()

,size,sum,mean,std
Asia,5.0,2.898666e+09,5.797333e+08,6.790979e+08
Australia,1.0,2.331602e+07,2.331602e+07,NaN
Europe,6.0,4.579297e+08,7.632161e+07,3.464767e+07
North America,2.0,3.528552e+08,1.764276e+08,1.996696e+08
South America,1.0,2.059153e+08,2.059153e+08,NaN
